# 👨‍💼👨‍🚀👨‍💻 Agent with Code Interpreter

This notebook is restricted to OpenAI API only

### Initialize the kernel

In [ ]:
#r "nuget: Microsoft.SemanticKernel, 1.15.0"
#r "nuget: Microsoft.SemanticKernel.Plugins.Core, 1.15.0-alpha"
#r "nuget: Microsoft.SemanticKernel.Plugins.Web, 1.15.0-alpha"
#r "nuget: Microsoft.SemanticKernel.Agents.Core, 1.15.0-alpha"
#r "nuget: Microsoft.SemanticKernel.Agents.OpenAI, 1.15.0-alpha"

#!import ../../Secrets.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.SemanticKernel.Agents;
using Microsoft.SemanticKernel.Agents.Chat;
using Microsoft.SemanticKernel.Agents.OpenAI;
using Microsoft.SemanticKernel.ChatCompletion;

var kernel = Kernel.CreateBuilder()
    // .AddAzureOpenAIChatCompletion(
    //     deploymentName: Secrets.DeploymentName,
    //     endpoint: Secrets.Endpoint,
    //     apiKey: Secrets.ApyKey)
    .AddOpenAIChatCompletion(
        modelId: Secrets.OpenAIModel,
        apiKey: Secrets.OpenAIApiKey)
    .Build();

### Let's create a chat group for agents and add a goal to the chat.

In [ ]:
#pragma warning disable SKEXP0110

AgentGroupChat chat = new();

var goal = """
    Jack Sparrow must solve the next equation: 4*x^2 + 2*x = 0
    Do not show how the equation was solved, only the final values.
    """;

chat.AddChatMessage(new ChatMessageContent(AuthorRole.User, goal));


### Create Jack Sparrow agent with file retrieval capabilities (agents are using the out-of-the-box RAG feature of the OpenAI assistants)

In [ ]:
#pragma warning disable SKEXP0110

var agent = await OpenAIAssistantAgent.CreateAsync(
    kernel: kernel,
    config: new OpenAIAssistantConfiguration(Secrets.OpenAIApiKey!),
    definition: new OpenAIAssistantDefinition
    {
        Instructions = """
            You are Jack Sparrow talking in Jack Sparrow style.
            Evaluate the context and reply providing exactly one meaningful dialog line.
            The dialog line must be only one sentence of maximum 50 words.
            """,
        Description = "A chat bot that replies to the message in the voice of Jack Sparrow talking style.",
        Name = "JackSparrow",
        Id = "JackSparrow_01",
        EnableCodeInterpreter = true,
        ModelId = Secrets.OpenAIModel
    });

### Now let's set EnableCodeInterpreter = true and rerun the notebook.

### ⚡ The chat is invoking Jack Sparrow agent

In [ ]:
#pragma warning disable SKEXP0001
#pragma warning disable SKEXP0110

Console.WriteLine($">>>>>>> {AuthorRole.User} > {goal}");

await foreach (var content in chat.InvokeAsync(agent))
{
    Console.WriteLine($"\n>>>>>>> {content.Role} [{content.AuthorName}] > {content.Content}");
}

### Notice that the agent is computing the missing information using code interpreter! (in our case is solving a quadratic equation).